In [1]:
import os
import geopandas as gpd
import pandas as pd
import os
import leafmap as leafmap
import folium
from folium import plugins

In [17]:
dolmenmap = "data/georeference/Dolmenwood_Players_Map_4326.tif"
hexmap = "data/hexmap_6mi_4326.geojson"

In [18]:
hex_gdf = gpd.read_file(hexmap).drop(
    columns=[
        "ogc_fid",
        "id",
        "left",
        "top",
        "right",
        "bottom",
        "row_index",
        "col_index",
    ]
)

hex_gdf["dolmen_id"].fillna("0000", inplace = True)

In [51]:
bbox_centroid = hex_gdf.dissolve().centroid

In [55]:
bbox_centroid.x

0   -73.768451
dtype: float64

In [21]:
def get_bounding_box(gdf_to_bound: gpd.GeoDataFrame) -> list:
    min_lon, min_lat, max_lon, max_lat = gdf_to_bound.total_bounds
    bbox = [(min_lat, min_lon), (max_lat, max_lon)]
    
    return bbox

bbox = get_bounding_box(gdf_to_bound=hex_gdf)
print(bbox)

[(40.18084591547344, -74.8790723687818), (41.31749689088554, -72.65111902750066)]


In [22]:
min_lon = bbox[0][1]
max_lon = bbox[1][1]
min_lat = bbox[0][0]
max_lat = bbox[1][0]

# m = folium.Map(
#     max_bounds=True,
#     location=[-20, -40],
#     zoom_start=6,
#     min_lat=min_lat,
#     max_lat=max_lat,
#     min_lon=min_lon,
#     max_lon=max_lon,
# )

In [2]:
style = {
    "stroke": True,
    "color": "#fefefe",
    "weight": 1,
    "opacity": 1,
    "fill": True,
    "fillColor": "#0000ff",
    "fillOpacity": 0,
}

hover_style = {"fillOpacity": 0.1, "weight": 2}

In [5]:
cog = "https://github.com/jackrosacker/dolmenwood-character-generator/blob/main/data/dolmenwood_map_COG.tif"

In [8]:
m = leafmap.Map("imageover")
m.add_tile_layer(url=cog, name="Dolmenwood Map", attribution="The Creator of Dolmenwood")
# m.add_raster(dolmenmap, layer_name="Dolmenwood")

# m.add_gdf(
#     gdf=hex_gdf,
#     layer_name="Hex Grid",
#     style=style,
#     hover_style=hover_style,
#     max_bounds=True,
#     zoom_start=6,
#     min_lat=min_lat,
#     max_lat=max_lat,
#     min_lon=min_lon,
#     max_lon=max_lon,
# )
# m.add_labels(
#     data=hex_gdf,
#     column="dolmen_id",
#     font_size="12pt",
#     font_color="white",
#     font_family="arial",
#     font_weight="bold",
# )

m

Map(center=[20, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text…

In [91]:
m = folium.Map(
    location=[bbox_centroid.y, bbox_centroid.x],
    zoom_start=9,
    min_zoom=9,
    tiles="https://api.maptiler.com/tiles/ec02ca7e-ebcc-48d5-9284-b1d4d36a61d6/{z}/{x}/{y}.png?key={}",
    attr="Dolmenwood Players Map",
    #max_zoom=8,
    min_lat=40,
    max_lat=41.5,
    min_lon=-75,
    max_lon=-72.5,
    max_bounds=True,
)

img = folium.raster_layers.ImageOverlay(
    name="Mercator projection SW",
    image=dolmenmap,
    bounds=bbox,
    opacity=0.6,
    interactive=True,
    cross_origin=False,
    zindex=1,
)

# img.add_to(m)
# folium.LayerControl().add_to(m)
plugins.MousePosition().add_to(m)
folium.GeoJson(hex_gdf).add_to(m)
# m.fit_bounds(m.get_bounds())

m

In [6]:
simple_hex_gdf.crs

<Projected CRS: EPSG:2263>
Name: NAD83 / New York Long Island (ftUS)
Axis Info [cartesian]:
- X[east]: Easting (US survey foot)
- Y[north]: Northing (US survey foot)
Area of Use:
- name: United States (USA) - New York - counties of Bronx; Kings; Nassau; New York; Queens; Richmond; Suffolk.
- bounds: (-74.26, 40.47, -71.8, 41.3)
Coordinate Operation:
- name: SPCS83 New York Long Island zone (US Survey feet)
- method: Lambert Conic Conformal (2SP)
Datum: North American Datum 1983
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich